In [65]:
import json
import pickle
from elasticsearch import Elasticsearch

In [7]:
es = Elasticsearch(hosts="iuni2.carbonate.uits.iu.edu:9200/", http_auth=('penghao', 'sooth_discolor'))

In [28]:
res = es.search(index="wos", body={
    "query": {
        "ids": {
            "values": ["WOS:A1985AUK0600019"]
        }
    }
})

In [29]:
print(json.dumps(res, indent=2))

{
  "took": 6348,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1,
      "relation": "eq"
    },
    "max_score": 1.0,
    "hits": [
      {
        "_index": "wos",
        "_type": "_doc",
        "_id": "WOS:A1985AUK0600019",
        "_score": 1.0,
        "_source": {
          "doc": {
            "EWUID": [
              {
                "WUID": {
                  "_coll_id": "WOS"
                },
                "edition": [
                  {
                    "_value": "WOS.SCI"
                  }
                ]
              }
            ],
            "UID": "WOS:A1985AUK0600019",
            "contributor": [
              [
                {
                  "name": {
                    "_r_id": "E-6817-2010",
                    "_role": "researcher_id",
                    "_seq_no": 1,
                    "display_name": "arosio, paolo",
           

In [62]:
# This only gives up to 10000 records.

# def es_iterate_all_documents(es, index, pagesize=250, **kwargs):
#     """
#     Helper to iterate ALL values from
#     Yields all the documents.
#     """
#     offset = 0
#     while True:
#         result = es.search(index=index, **kwargs, body={
#             "size": pagesize,
#             "from": offset
#         })
#         hits = result["hits"]["hits"]
#         # Stop after no more docs
#         if not hits:
#             break
#         # Yield each entry
#         yield from (hit['_source'] for hit in hits)
#         # Continue from there
#         offset += pagesize

def es_iterate_all_documents(es, index, pagesize=250, scroll_timeout="1m", **kwargs):
    """
    Helper to iterate ALL values from a single index
    Yields all the documents.
    """
    is_first = True
    while True:
        # Scroll next
        if is_first: # Initialize scroll
            result = es.search(index=index, scroll="1m", **kwargs, body={
                "size": pagesize
            })
            is_first = False
        else:
            result = es.scroll(body={
                "scroll_id": scroll_id,
                "scroll": scroll_timeout
            })
        scroll_id = result["_scroll_id"]
        hits = result["hits"]["hits"]
        # Stop after no more docs
        if not hits:
            break
        # Yield each entry
        yield from (hit['_source'] for hit in hits)

In [66]:
with open('./dois_mentioned.pickle', 'rb') as ofile:
    dois = pickle.load(ofile)

In [67]:
len(dois)

279504

In [ ]:
cn = 0
pos = 0
with open('./doi_wos_authors.json', 'w') as ofile:
    for entry in es_iterate_all_documents(es, 'wos'):
        cn += 1
        if 'identifier' in entry['doc'] and 'name' in entry['doc']:
            mdict = {item['_type']: item['_value'] for item in entry['doc']['identifier'][0]}
            if 'doi' in mdict:
                doi = mdict['doi'].lower()
                if doi in dois:
                    authors = entry['doc']['name'][0]
                    for aut in authors:
                        if '_reprint' in aut:
                            pos += 1
                            row = {'doi_lowercase': doi, 'authors': authors}
                            ofile.write(json.dumps(row) + '\n')
                            break
        if cn%100000 == 0:
            print('searched %d documents, saved %d positive docs...'%(cn, pos))

searched 100000 documents, saved 783 positive docs...
searched 200000 documents, saved 1527 positive docs...
searched 300000 documents, saved 1803 positive docs...
searched 400000 documents, saved 2982 positive docs...
searched 500000 documents, saved 4235 positive docs...
searched 600000 documents, saved 5279 positive docs...
searched 700000 documents, saved 5337 positive docs...
searched 800000 documents, saved 5831 positive docs...
searched 900000 documents, saved 5921 positive docs...
searched 1000000 documents, saved 6600 positive docs...
searched 1100000 documents, saved 7861 positive docs...
searched 1200000 documents, saved 8543 positive docs...
searched 1300000 documents, saved 9770 positive docs...
searched 1400000 documents, saved 11039 positive docs...
searched 1500000 documents, saved 11955 positive docs...
searched 1600000 documents, saved 12677 positive docs...
searched 1700000 documents, saved 13364 positive docs...
searched 1800000 documents, saved 13985 positive docs.

searched 14500000 documents, saved 78993 positive docs...
searched 14600000 documents, saved 79137 positive docs...
searched 14700000 documents, saved 79321 positive docs...
searched 14800000 documents, saved 79458 positive docs...
searched 14900000 documents, saved 79458 positive docs...
searched 15000000 documents, saved 79458 positive docs...
searched 15100000 documents, saved 79647 positive docs...
searched 15200000 documents, saved 79830 positive docs...
searched 15300000 documents, saved 80370 positive docs...
searched 15400000 documents, saved 80447 positive docs...
searched 15500000 documents, saved 80586 positive docs...
searched 15600000 documents, saved 80929 positive docs...
searched 15700000 documents, saved 80956 positive docs...
searched 15800000 documents, saved 80978 positive docs...
searched 15900000 documents, saved 80978 positive docs...
searched 16000000 documents, saved 80996 positive docs...
searched 16100000 documents, saved 81013 positive docs...
searched 16200

searched 29000000 documents, saved 92807 positive docs...
searched 29100000 documents, saved 92853 positive docs...
searched 29200000 documents, saved 93155 positive docs...
searched 29300000 documents, saved 93514 positive docs...
searched 29400000 documents, saved 93566 positive docs...
searched 29500000 documents, saved 93566 positive docs...
searched 29600000 documents, saved 93566 positive docs...
searched 29700000 documents, saved 93566 positive docs...
searched 29800000 documents, saved 93566 positive docs...
searched 29900000 documents, saved 93815 positive docs...
searched 30000000 documents, saved 93815 positive docs...
searched 30100000 documents, saved 94312 positive docs...
searched 30200000 documents, saved 95043 positive docs...
searched 30300000 documents, saved 95821 positive docs...
searched 30400000 documents, saved 95823 positive docs...
searched 30500000 documents, saved 95824 positive docs...
searched 30600000 documents, saved 95858 positive docs...
searched 30700

searched 43000000 documents, saved 109477 positive docs...
searched 43100000 documents, saved 109503 positive docs...
searched 43200000 documents, saved 109514 positive docs...
searched 43300000 documents, saved 109571 positive docs...
searched 43400000 documents, saved 109642 positive docs...
searched 43500000 documents, saved 109658 positive docs...
searched 43600000 documents, saved 109668 positive docs...
searched 43700000 documents, saved 109670 positive docs...
searched 43800000 documents, saved 109676 positive docs...
searched 43900000 documents, saved 109699 positive docs...
searched 44000000 documents, saved 109711 positive docs...
searched 44100000 documents, saved 109729 positive docs...
searched 44200000 documents, saved 109735 positive docs...
searched 44300000 documents, saved 109768 positive docs...
searched 44400000 documents, saved 109791 positive docs...
searched 44500000 documents, saved 109819 positive docs...
searched 44600000 documents, saved 109874 positive docs.

searched 56900000 documents, saved 126117 positive docs...
searched 57000000 documents, saved 126369 positive docs...
searched 57100000 documents, saved 126683 positive docs...
searched 57200000 documents, saved 126993 positive docs...
searched 57300000 documents, saved 127093 positive docs...
searched 57400000 documents, saved 127315 positive docs...
searched 57500000 documents, saved 127496 positive docs...
searched 57600000 documents, saved 127774 positive docs...
searched 57700000 documents, saved 128031 positive docs...
searched 57800000 documents, saved 128143 positive docs...
searched 57900000 documents, saved 128420 positive docs...
searched 58000000 documents, saved 128723 positive docs...
searched 58100000 documents, saved 129169 positive docs...
searched 58200000 documents, saved 129449 positive docs...
searched 58300000 documents, saved 129701 positive docs...
searched 58400000 documents, saved 129952 positive docs...
searched 58500000 documents, saved 130211 positive docs.

In [79]:
print('searched %d documents in total, saved %d positive docs...'%(cn, pos))

searched 72873288 documents in total, saved 217010 positive docs...


In [71]:
! ls -alhS

total 412M
-rw-rw-r--  1 penghao penghao 403M Nov 17 22:20 doi_wos_authors.json
-rw-r--r--  1 penghao penghao 9.1M Nov 17 12:13 dois_mentioned.pickle
-rw-rw-r--  1 penghao penghao  73K Nov 17 21:19 get_wos_authors.ipynb
drwxrwxr-x  3 penghao penghao  32K Nov 17 21:19 .
drwx------ 21 penghao penghao  32K Nov 16 17:38 ..
drwxrwxr-x  2 penghao penghao  32K Nov 17 12:18 .ipynb_checkpoints
-rw-rw-r--  1 penghao penghao  14K Nov 17 10:38 jupyter_to_elasticsearch_connection.ipynb
-rw-rw-r--  1 penghao penghao  751 Nov 17 10:39 cli_queries
-rw-rw-r--  1 penghao penghao  393 Nov 17 10:39 kibana_queries
